In [80]:
import requests
import hashlib
import hmac
import base64
import urllib.parse
import time

class Kraken(object):
      
    def __init__(self,key = '',secret = ''):
        self.key = key
        self.secret = secret
        self.url = 'https://api.kraken.com'
        self.apiv = '0'
        self.pp = ('public','private')
        self.method = {'gst':'Time',
                       'gai':'Assets',
                       'gtap':'AssetPairs',
                       'gti':'Ticker',
                       'god':'OHLC',
                       'gob':'Depth',
                       'grt':'Trades',
                       'grsd':'Spread',
                       'gab':'Balance',
                       'gtb':'TradeBalance',
                       'goo':'OpenOrders',
                       'gco':'ClosedOrders',
                       'qoi':'QueryOrders',
                       'gth':'TradesHistory',
                       'gti2':'QueryTrades',
                       'gop':'OpenPositions',
                       'gli':'Ledgers',
                       'ql':'QueryLedgers',
                       'gtv':'TradeVolume',
                       'aso':'AddOrder',
                       'coo':'CancelOrder'}
        
    def get_keys(self,path):
        f = open(path,"r")
        self.key = f.readline().strip()
        self.secret = f.readline().strip()
    
    #define public methods
    def get_server_time(self):
        return requests.get(self.url+'/'+self.apiv+'/'+self.pp[0]+'/'+self.method['gst'])
    
    
    def get_asset_info(self, info=None, aclass=None, asset=None):
        if asset == None:
            payload = {'info': info, 'aclass': aclass, 'asset': asset}
        else:
            payload = {'info': info, 'aclass': aclass, 'asset': ','.join(asset[:])}
        return requests.get(self.url+'/'+self.apiv+'/'+self.pp[0]+'/'+self.method['gai'], params=payload)
        

    def get_tradable_asset_pairs(self, info=None, pair=None):
        if pair == None:
            payload = {'info': info, 'pair': pair}
        else:
            payload = {'info': info, 'pair': ','.join(pair[:])}
        return requests.get(self.url+'/'+self.apiv+'/'+self.pp[0]+'/'+self.method['gtap'], params=payload)
    
    
    def get_ticker_information(self, pair):
        payload = {'pair': ','.join(pair[:])}
        return requests.get(self.url+'/'+self.apiv+'/'+self.pp[0]+'/'+self.method['gti'], params=payload)
    
    
    def get_OHLC_data(self, pair, interval=None, since=None):
        payload = {'pair': pair,'interval': interval, 'since': since}
        return requests.get(self.url+'/'+self.apiv+'/'+self.pp[0]+'/'+self.method['god'], params=payload)
    
    
    def get_order_book(self, pair, count=None):
        payload = {'pair': pair,'count':count}
        return requests.get(self.url+'/'+self.apiv+'/'+self.pp[0]+'/'+self.method['gob'], params=payload)
    
    #WIP
    def get_recent_trades(self, pair, since=None):
        payload = {'pair': pair, 'since': since}
        return requests.get(self.url+'/'+self.apiv+'/'+self.pp[0]+'/'+self.method['grt'], params=payload)
    
    
    #WIP
    def get_recent_spread_data(self, pair, since=None):
        payload = {'pair': pair, 'since': since}
        return requests.get(self.url+'/'+self.apiv+'/'+self.pp[0]+'/'+self.method['grsd'], params=payload)
    
    
    #define private methods
    '''
        HTTP Header
            API-Key = API key
            API-Sign = Message signature using HMAC-SHA512 of (URI path + SHA256(nonce + POST data)) and base64 decoded secret API key
        POST data:
            nonce = always increasing unsigned 64 bit integer
            otp = two-factor password (if two-factor enabled, otherwise not required)
    '''
    
    def get_account_balance(self):
        
        # get input, nonce and postdata
        req['nonce'] = int(1000*time.time())
        postdata = urllib.parse.urlencode(req)
        
        #encode nonce+POST data
        encoded = (str(req['nonce']) + postdata).encode()

        # message = URI path + SHA256(nonce + POST data)
        urlpath = '/'+self.apiv+'/'+self.pp[1]+'/'+self.method['gab']
        message = urlpath.encode() + hashlib.sha256(encoded).digest()

        # get signature and signature digest
        signature = hmac.new(base64.b64decode(self.secret), message, hashlib.sha512)
        sigdigest = base64.b64encode(signature.digest())

        #define headers data
        headers = {'API-Key': self.key,'API-Sign': sigdigest.decode()}
        
        return requests.post(self.url+'/'+self.apiv+'/'+self.pp[1]+'/'+self.method['gab'], data=req, headers=headers)
    
    #WIP
    def get_trade_balance(self):
        pass
    
    #WIP
    def get_open_orders(self):
        pass
    
    #WIP
    def get_closed_orders(self):
        pass
    
    #WIP
    def query_orders_info(self):
        pass
    
    #WIP
    def get_trades_history(self):
        pass
    
    #WIP
    def query_trades_info(self):
        pass
    
    #WIP
    def get_open_positions(self):
        pass
    
    #WIP
    def get_ledgers_info(self):
        pass
    
    #WIP
    def query_ledgers(self, id):
        #NOTE: Can only query a max on 20
        
        # get input, nonce and postdata
        req = {'id': id, 'nonce': int(1000*time.time())}
        postdata = urllib.parse.urlencode(req)
        
        #encode nonce+POST data
        encoded = (str(req['nonce']) + postdata).encode()

        # message = URI path + SHA256(nonce + POST data)
        urlpath = '/'+self.apiv+'/'+self.pp[1]+'/'+self.method['ql']
        message = urlpath.encode() + hashlib.sha256(encoded).digest()

        # get signature and signature digest
        signature = hmac.new(base64.b64decode(self.secret), message, hashlib.sha512)
        sigdigest = base64.b64encode(signature.digest())

        #define headers data
        headers = {'API-Key': self.key,'API-Sign': sigdigest.decode()}
        
        return requests.post(self.url+'/'+self.apiv+'/'+self.pp[1]+'/'+self.method['ql'], data=req, headers=headers)
    
    #WIP
    def get_trade_volume(self):
        pass
    
    #WIP
    def add_standard_order(self):
        pass
    
    #WIP
    def cancel_open_order(self):
        pass
    
    
    
    

In [81]:
k = Kraken()

In [82]:
k.get_keys('C:\\test.txt')

In [83]:
r = k.query_ledgers('L3IFP6-JT2SB-3DUJUR')

In [78]:
r.json()['result']

{'L3IFP6-JT2SB-3DUJUR': {'aclass': 'currency',
  'amount': '22000.00000000',
  'asset': 'XXDG',
  'balance': '22000.00000000',
  'fee': '0.00000000',
  'refid': 'QSBFFDR-IRX5ML-LIQD2F',
  'time': 1470489605.0162,
  'type': 'deposit'}}

In [79]:
'''
XDAOXETH,XDAOXXBT  example pairs
XXDG  example asset
L3IFP6-JT2SB-3DUJUR example ledger entry
'''

'\nXDAOXETH,XDAOXXBT  example pairs\nXXDG  example asset\nL3IFP6-JT2SB-3DUJUR example ledger entry\n'